In [ ]:

import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_capability())

In [ ]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [ ]:
import tensorflow as tf
print(tf.__version__)

2.1.0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!cp "/content/drive/My Drive/koulu_honmia/kandi19/IDA.zip" /content/

In [ ]:
!cp "/content/drive/My Drive/koulu_honmia/kandi19/benthic/loadbm.py" /content/

In [ ]:
%%capture
!unzip IDA.zip

In [ ]:
!rm IDA.zip

In [ ]:
import numpy as np
import os
import ntpath
import platform
import matplotlib.pyplot as plt
from loadbm import create_df, create_tf_dataset, prepare_for_training


datapath = 'IDA/Separate lists with numbering/Machine learning splits'
img_path = 'IDA/Images/'

split = 1

train_fname = 'train'+str(split)+'.txt'
test_fname = 'test'+str(split)+'.txt'
val_fname = 'val'+str(split)+'.txt'

part_dat = False

df_train = create_df(os.path.join(datapath, train_fname),
                     img_path,
                     partial_dataset=part_dat,
                     seed=123)

df_test = create_df(os.path.join(datapath, test_fname),
                     img_path,
                     partial_dataset=part_dat,
                     seed=123)

df_val = create_df(os.path.join(datapath, val_fname),
                     img_path,
                     partial_dataset=part_dat,
                     seed=123)

from sklearn.utils import shuffle

df_train = shuffle(df_train)
df_val = shuffle(df_val)

df_train.head()

,path,label
202496,IDA/Images/Micrasema_gel85/0-Micrasema_gel85.1...,23
89122,IDA/Images/Ephemerella_muc258/1-Ephemerella_mu...,10
152601,IDA/Images/Lepidostoma_hir177/1-Lepidostoma_hi...,18
193845,IDA/Images/Limnius_vol64/1-Limnius_vol64.1.201...,22
37388,IDA/Images/Baetis_rho62/1-Baetis_rho62.1.2016-...,4


In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
IMSIZE = (224,224,3)
BATCH_SIZE = 32

train_ds = create_tf_dataset(df_train, imsize=IMSIZE, onehot=True)

val_ds = create_tf_dataset(df_val, imsize=IMSIZE, onehot=True)


train_ds = prepare_for_training(train_ds, 
                                shuffle_buffer_size=1000,
                                batch_size=BATCH_SIZE)

val_ds = prepare_for_training(val_ds, 
                              shuffle_buffer_size=1000,
                              batch_size=BATCH_SIZE)

for image, label in train_ds.take(5):
    print(image.shape)
    print(label.shape)


(32, 224, 224, 3)
(32, 39)
(32, 224, 224, 3)
(32, 39)
(32, 224, 224, 3)
(32, 39)
(32, 224, 224, 3)
(32, 39)
(32, 224, 224, 3)
(32, 39)


In [ ]:
import tensorflow.keras.backend as K
def focal_loss(gamma=2.0, alpha=0.25):

    def loss(y,yhat):
        
      y = K.cast(y, tf.float32)
      yhat = tf.convert_to_tensor(yhat, dtype=y.dtype)

      ce = -K.sum(y*K.log(yhat+1e-7), axis=1, keepdims=True)
      
      p_t = (y * yhat)
      alpha_factor = (y * alpha)
      
      modulating_factor = ((1.0 - p_t)**gamma)
      
      fl = K.sum(alpha_factor*modulating_factor*ce, axis=-1)
  
      return fl

    return loss

alpha = np.histogram(df_train['label'].values,bins=39)[0]
alpha = 1-(alpha/np.sum(alpha))

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D


def get_pretrained(imsize=(224, 224, 3), classes=39):
        base_model = InceptionV3(input_shape = imsize, 
                                 weights='imagenet', 
                                 include_top=False)
        
        base_model.trainable = True
        
        x = base_model.output
        x = GlobalAveragePooling2D()(x)
        x = Dense(256, activation='relu')(x)
        predictions = Dense(classes, activation='softmax')(x)
        
        model = Model(inputs=base_model.input, outputs=predictions)
        
        return model

model = get_pretrained()

loss = focal_loss(gamma=1.0, alpha=alpha) # !!!!!! Gamma -> 1.0

model.compile(optimizer = 'adam', loss = loss,
                  metrics=['accuracy'])

87916544/87910968 [==============================] - 6s 0us/step


In [ ]:
from tensorflow.keras.callbacks import CSVLogger
import datetime


tr_steps = len(df_train)//BATCH_SIZE
val_steps = len(df_val)//BATCH_SIZE

model.fit(train_ds, 
          validation_data= val_ds, 
          steps_per_epoch= tr_steps, 
          epochs = 10,
          validation_steps = val_steps)


Train for 10043 steps, validate for 1434 steps
Epoch 1/10
10043/10043 [==============================] - 1610s 160ms/step - loss: 0.3398 - accuracy: 0.8490 - val_loss: 0.5042 - val_accuracy: 0.8058
Epoch 2/10
10043/10043 [==============================] - 1594s 159ms/step - loss: 0.0788 - accuracy: 0.9575 - val_loss: 0.3568 - val_accuracy: 0.8713
Epoch 3/10
10043/10043 [==============================] - 1588s 158ms/step - loss: 0.0440 - accuracy: 0.9761 - val_loss: 0.5290 - val_accuracy: 0.8410
Epoch 4/10
10043/10043 [==============================] - 1588s 158ms/step - loss: 0.0310 - accuracy: 0.9830 - val_loss: 0.3752 - val_accuracy: 0.8843
Epoch 5/10
10043/10043 [==============================] - 1588s 158ms/step - loss: 0.0236 - accuracy: 0.9871 - val_loss: 0.4096 - val_accuracy: 0.8809
Epoch 6/10
 7202/10043 [====================>.........] - ETA: 7:08 - loss: 0.0204 - accuracy: 0.9890Buffered data was truncated after reaching the output size limit.

In [ ]:
from tensorflow.keras.optimizers import Adam
adam = Adam(learning_rate=0.0001) #original 0.001

from tensorflow.keras.callbacks import LearningRateScheduler
def scheduler(epoch):
    return 0.0001 * tf.math.exp(0.1 * (-epoch))

lr_cb = LearningRateScheduler(scheduler)

model.compile(optimizer = adam, loss = loss,
                  metrics=['accuracy'])

model.fit(train_ds, 
          validation_data= val_ds, 
          steps_per_epoch= tr_steps, 
          epochs = 5,
          validation_steps = val_steps,
          callbacks=[lr_cb])

Train for 10043 steps, validate for 1434 steps
Epoch 1/5
10043/10043 [==============================] - 1603s 160ms/step - loss: 0.0017 - accuracy: 0.9991 - val_loss: 0.2616 - val_accuracy: 0.9265
Epoch 2/5
10043/10043 [==============================] - 1603s 160ms/step - loss: 4.9371e-04 - accuracy: 0.9997 - val_loss: 0.2830 - val_accuracy: 0.9249
Epoch 3/5
10043/10043 [==============================] - 1599s 159ms/step - loss: 2.0808e-04 - accuracy: 0.9999 - val_loss: 0.2877 - val_accuracy: 0.9285
Epoch 4/5
10043/10043 [==============================] - 1608s 160ms/step - loss: 2.0182e-04 - accuracy: 0.9999 - val_loss: 0.2865 - val_accuracy: 0.9287
Epoch 5/5
10043/10043 [==============================] - 1599s 159ms/step - loss: 1.8647e-04 - accuracy: 0.9999 - val_loss: 0.2861 - val_accuracy: 0.9299


In [ ]:
model.save('32-xx-2020.h5')

In [ ]:
!cp '32-xx-2020.h5' "/content/drive/My Drive/koulu_honmia/kandi19/"

Evaluation:

In [ ]:
test_ds = create_tf_dataset(df_test, imsize=IMSIZE, onehot=True)
test_ds = test_ds.batch(BATCH_SIZE)

In [ ]:
!cp "/content/drive/My Drive/koulu_honmia/kandi19/benthic/combine_insects.py" /content/

In [ ]:
preds = model.predict(test_ds, verbose=True)
yhat = np.argmax(preds,axis=1)+1
y_test = df_test['label']

acc = np.sum(yhat==y_test)/len(y_test)
print('Image accuracy: {:.4f}'.format(acc))

np.save('32-xx-2020.npy', preds)

2897/2897 [==============================] - 150s 52ms/step
Image accuracy: 0.9253


In [ ]:
!cp '32-xx-2020.npy' "/content/drive/My Drive/koulu_honmia/kandi19/"

In [ ]:
#%% Insect combine
from combine_insects import add_insect_class, add_yhat

df_test_preds = add_insect_class(df_test)

# adding predictions to dataframe for insect-wise prediction
df_test_preds = add_yhat(df_test_preds,yhat)

dfg = df_test_preds.groupby(['label','insect'],as_index=False)['pred'].agg(lambda x:x.value_counts().index[0])

acc_g = np.sum(dfg['pred']==dfg['label'])/len(dfg)
print('Aggregate accuracy: {:.4f}'.format(acc_g))

Aggregate accuracy: 0.9565


In [ ]:
model.history.history

{'accuracy': [0.9990572, 0.9997044, 0.99987555, 0.99988484, 0.99987245],
 'loss': [0.0017169833088054264,
  0.0004937130546234563,
  0.00020808415792836207,
  0.00020181658747020102,
  0.00018647123881658953],
 'lr': [1e-04, 9.0483736e-05, 8.187308e-05, 7.4081814e-05, 6.7032e-05],
 'val_accuracy': [0.9265385, 0.92486054, 0.92852163, 0.92873955, 0.9299163],
 'val_loss': [0.26161028114398693,
  0.2829957124765925,
  0.28767670817209534,
  0.28652760656972937,
  0.2860776092174647]}